<a href="https://colab.research.google.com/github/Bryan-Az/Advanced-Prompt-Engineering/blob/main/notebooks/prompt_techniques/RASCEF_REACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applying RASCEF and REACT to Prompt Template Generation

## Imports and Installs

In [1]:
%%capture
!pip install datasets

In [2]:
import json
import datasets
import openai
from google.colab import userdata
import os

## Loading the Sample Templates

In [6]:
%%capture
# downloading the react sample json from github
! wget https://raw.githubusercontent.com/Bryan-Az/Advanced-Prompt-Engineering/refs/heads/main/prompts/ReACT_sample_structure.json -O ReACT_sample_structure.json
! wget https://raw.githubusercontent.com/Bryan-Az/Advanced-Prompt-Engineering/refs/heads/main/prompts/RASCEF_sample_structure.json -O RASCEF_sample_structure.json

In [7]:
#loading the sample templates as json
with open('ReACT_sample_structure.json', 'r') as file:
    ReACT_sample_structure = json.load(file)
with open('RASCEF_sample_structure.json', 'r') as file:
    RASCEF_sample_structure = json.load(file)

In [5]:
display(ReACT_sample_structure)

{'prompt': '[Initial question or task]',
 'actions': [{'name': '[Action Name]',
   'description': '[Brief description of what the action does]',
   'result_format': '[Format of the result]'}],
 'action_requests': [{'condition': '[Condition for triggering the action]',
   'requested_action': '[Name of the action to perform]'}]}

In [9]:
display(RASCEF_sample_structure)

{'role': 'Expert',
 'action': 'Analyze and [Desired Outcome]',
 'steps': ['1. [Step 1 Description]',
  '2. [Step 2 Description]',
  '3. [Step 3 Description]'],
 'context': '[Brief description of the situation or data]',
 'examples': [{'input_data': '[Example input format]',
   'expected_output': '[Desired output format]'}],
 'format': '[How the output should be structured]'}

## Initializing the OpenAI Model API Class

In [10]:
openai_key = userdata.get("openai")
os.environ["OPENAI_API_KEY"] = openai_key

In [11]:
model_name = 'gpt-3.5-turbo'
def get_completion(prompt, model=model_name):
  """Generates text using the OpenAI API.

  Args:
    prompt: The prompt to send to the API.
    model: The name of the model to use.

  Returns:
    The generated text.
  """
  response = openai.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": prompt}],
      temperature=0.01
  )
  return response.choices[0].message.content

## Using the Model API to Generate ICL Prompts for 3 Domains
As I already generated prompt templates in the fields of healthcare, software, math, finance, and data analysis - I will ask the model to generate 8 more prompt templates in distinct fields for both RASCEF and ReACT strategies.

In [98]:
# calling the OpenAI API to use this template to generate RASCEF prompts
RASCEF_prompt_generation_prompt = f"Can you help me use this sample json template to generate 8 RASCEF style JSON prompts in the fields of construction, sculpting, gardening, farming, environmental science, and law enforcement?. Only return the JSON, avoid seperating the JSONs into seperate outer nests - do not add bulleted list. Template: {RASCEF_sample_structure}"

rascef_response = get_completion(RASCEF_prompt_generation_prompt)

In [99]:
rascef_response

'{\n  "role": "Expert",\n  "action": "Analyze and optimize construction techniques for sustainable building practices",\n  "steps": [\n    "1. Research current construction methods and materials",\n    "2. Identify areas for improvement in sustainability",\n    "3. Develop and test new construction techniques"\n  ],\n  "context": "Looking to reduce environmental impact and increase energy efficiency in construction projects",\n  "examples": [\n    {\n      "input_data": "Data on current construction practices and materials",\n      "expected_output": "Recommendations for sustainable building practices"\n    }\n  ],\n  "format": "Report detailing findings and recommendations"\n}\n\n{\n  "role": "Expert",\n  "action": "Analyze and refine sculpting techniques for public art installations",\n  "steps": [\n    "1. Study different sculpting methods and materials",\n    "2. Experiment with new techniques to achieve desired artistic effects",\n    "3. Collaborate with artists and designers to 

In [100]:
# calling the OpenAI API to use this template to generate ReACT prompts
react_prompt_generation_prompt= f"Can you help me use this sample json template to generate 8 ReACT style JSON prompts for Management, Fashion, Food/Beverage, Healthcare, Journalism, Library Science, Music Therapy, and Social Work? Only return the JSON, avoid seperating the JSONs into seperate outer nests - do not add bulleted list. Template: {ReACT_sample_structure}"
react_response = get_completion(react_prompt_generation_prompt)

In [101]:
# using regex to select the json from the text output
import re
def extract_json_from_text(text):
    json_objects = []
    while True:  # Iterate until no more JSON objects are found
        # find many instances of json objects at any level
        matches = re.search(r'({.*?})', text, re.DOTALL)
        if matches is None:
            break  # No more JSON objects, exit loop
        json_str = matches.group(1)
        text = text[len(json_str):]  # Remove the processed JSON string from text
        try:
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except:
            pass  # Invalid JSON, ignore
    return json_objects
# applying it on the 2 prompt styles
prompts = [react_response, rascef_response]
extracted_rascef_prompts = extract_json_from_text(prompts[1])
extracted_react_prompts = extract_json_from_text(prompts[0])

In [102]:
print(len(extracted_rascef_prompts))
print(len(extracted_react_prompts))

2
10


In [103]:
# writing the output for both rascef and react prompts by writing to a new json file at prompts/generated_prompts
with open('prompts/generated_prompts/2_rascef_prompts.json', 'w') as file:
    json.dump(extracted_rascef_prompts, file, indent=4)
with open('prompts/generated_prompts/10_react_prompts.json', 'w') as file:
    json.dump(extracted_react_prompts, file, indent=4)